In [9]:
import healpy as hp
import numpy as np
import s2wav
import s2wav.filters as filters

# Step 1: Read the Healpix map
healpix_map_path = 'data/planck_simulation/ffp10_newdust_total_030_full_map.fits'
healpix_map = hp.read_map(healpix_map_path)

# Step 2: Convert Healpix map to spherical harmonic coefficients
L_max = hp.npix2nside(len(healpix_map))*2  # Define maximum multipole moment (choose appropriately based on your data)
alm = hp.map2alm(healpix_map, lmax=L_max-1)

# Step 3: Rearrange coefficients for s2wav
f = np.zeros((L_max, 2 * L_max - 1), dtype=np.complex128)

for l in range(L_max):
    for m in range(-l, l + 1):
        index = hp.Alm.getidx(L_max - 1, l, abs(m))
        if m < 0:
            f[l, L_max + m - 1] = (-1)**m * np.conj(alm[index])
        else:
            f[l, L_max + m - 1] = alm[index]

# Step 4: Perform wavelet analysis
N = 1  # Number of directional wavelets
filter = filters.filters_directional_vectorised(L_max, N)

# wavelet_coeffs, scaling_coeffs
f_wav, f_scal = s2wav.analysis(f, L_max, N, filters=filter, reality=True)

# Step 5: Evaluate results or reconstruct the signal if needed

In [48]:
f.shape

(2048, 4095)

In [42]:
type(f_scal[1])

jaxlib.xla_extension.ArrayImpl

In [47]:
f_scal

Array([[7.68547842e-11+0.j]], dtype=complex128)

In [55]:
import jax.numpy as jnp
import numpy as np

# Assuming f_wav is a list of jaxlib.xla_extension.ArrayImpl objects

# Convert each element in f_wav to a numpy array and save it
for i, wav in enumerate(f_wav):
    np_wav = np.array(wav)  # Convert JAX array to numpy array
    np.save(f"/Users/maxwang/Documents/projects/CMB_plot/convolution/30/f_wav_{i}", np_wav)  # Save numpy array to a file

# Convert each element in f_scale to a numpy array and save it

np_scal = np.array(f_scal)  # Convert JAX array to numpy array
np.save(f"/Users/maxwang/Documents/projects/CMB_plot/convolution/30/f_scal", np_scal)  # Save numpy array to a file

In [ ]:
for i, wav in enumerate(f_wav):
    np_wav = np.array(wav)  # Convert JAX array to numpy array
    np.save(f"/convolution/30/f_wav_{i}.npy", np_wav)  # Save numpy array to a file


In [38]:
wavelet_coeffs = np.array([np.array(wavelet_coefficents) for wavelet_coefficents in f_wav])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (12, 1) + inhomogeneous part.

In [37]:
for i, wav in enumerate(f_wav):
    np_wav = np.array(wav)
    wavelet_coeffs= np.append(wavelet_coeffs,np_wav)

wavelet_coeffs

TypeError: 'int' object is not callable

In [29]:
f_wav[0]-np.load("convolution/f_wav_0.npy")

Array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]], dtype=float64)

In [28]:
# # Convert f_wav to a numpy array and stack them if compatible
# np_f_wav = np.array([np.array(wav) for wav in f_wav])

# # Save the stacked array
# np.savez_compressed("convolution/f_30_wav_combined.npy", np_f_wav)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (12, 1) + inhomogeneous part.

In [19]:
F_wav_30_NP = np.asanyarray(f_wav)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (12, 1) + inhomogeneous part.

In [11]:
np.save("/Users/maxwang/Documents/projects/CMB_plot/convolution/f_wav_30",F_wav_30_NP)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (12, 1) + inhomogeneous part.

In [30]:
f_check = s2wav.synthesis(f_wav, f_scal, L_max, N, filters=filter, reality=True)


In [58]:
reconstructed_map = np.array(f_check)
np.save(f"/Users/maxwang/Documents/projects/CMB_plot/convolution/30/reconstructed_map", reconstructed_map)  # Save numpy array to a file

In [61]:
load_map = np.load("/Users/maxwang/Documents/projects/CMB_plot/convolution/30/reconstructed_map.npy")

np.nanmean(np.abs(load_map-f))

2.0528992438406644e-07

In [39]:
print(f"Mean absolute error = {np.nanmean(np.abs(f_check - f))}")

Mean absolute error = 2.0528992438406644e-07
